In [165]:
# This is to collect important functions used in quantum field theory

# Importing libraries
import numpy as np
import scipy as sp
import scipy.integrate as spi
import cmath as cm
import sympy as sym
import warnings
import itertools
from types import FunctionType

# define the minkowski (psuedo-)metric
def minkowski(x : np.matrix, y : np.matrix) -> float:
    '''
    ----------
    Parameters
    ----------
    x, y : 4D array-like
        column vectors of shape (4,1)
    ----------
    Output
    ----------
    The Minkowski inner product of x and y
    '''
    return x[0,0]*y[0,0] - x[1,0]*y[1,0] - x[2,0]*y[2,0] - x[3,0]*y[3,0]

# Define the Pauli matrices
def sigma(n: int | np.matrix) -> np.matrix:
      """ 
    ----------
    Parameters
    ----------
    n : either int or array-like of shape (4,)
        if n is an integer, it should be in {0,1,2,3}
        if n is array-like, it should be a 4D row or column vector
    ----------
    Output
    ----------
    if n is an integer, it returns the n-th Pauli matrix
    if n is an array, it returns the Pauli vector corresponding to the array
    """
      if type(n) == int:
        if n == 0:
            return np.matrix([[1, 0], [0, 1]])
        elif n == 1:
            return np.matrix([[0, 1], [1, 0]])
        elif n == 2:
            return np.matrix([[0, -1j], [1j, 0]])
        elif n == 3:
            return np.matrix([[1, 0], [0, -1]])
        else:
            raise ValueError('n must be 0, 1, 2, or 3')
      else: # n is an array-like
            m = np.array(n)
            if m.shape == (4,1):
                return m[0][0]*sigma(0) + m[1][0]*sigma(1) + m[2][0]*sigma(2) + m[3][0]*sigma(3)
            elif m.shape == (1,4):
                return m[0][0]*sigma(0) + m[0][1]*sigma(1) + m[0][2]*sigma(2) + m[0][3]*sigma(3)
            else:   
                raise ValueError('sigma input must be integer of 4D array-like')
            
# define the inverse of sigma            
def sigma_inv(A : np.matrix) -> np.matrix:
    '''
    ----------
    Parameters
    ----------

    A : 2x2 matrix
        
    ----------
    Output
    ----------

    The 4D vector x such that sigma(x) = A
    If A = A^* is a Hermitian matrix, then x is a real vector

    '''
    a = A[0,0]; b=A[0,1]; c=A[1,0]; d=A[1,1]
    x = 1 /2 * np.matrix([ [a+d], [b+c], [1j*(b-c)], [a-d] ])
    return x
        


# Define Gamma matrices (chiral form by default)
def gamma(n : int , repr : str = 'chiral') -> np.matrix:
    '''
    ----------
    Parameters
    ----------
    n : int
        The index of the gamma matrix. n = 0, 1, 2, 3, 5
    repr : str
        The representation of the gamma matrices. 'chiral', 'dirac', or 'majorana'. 
        Default is 'chiral'.
    ----------
    Output
    ----------
    The n-th gamma matrix in the specified representation.

    '''
    if repr == 'chiral':
        if n == 0:
            return np.matrix([[0,0,1,0], [0,0,0,1], [1,0,0,0], [0,1,0,0]])
        elif n == 1:
            return np.matrix([[0,0,0,1],[0,0,1,0],[0,-1,0,0],[-1,0,0,0]])
        elif n == 2:
            return np.matrix([[0,0, 0,-1j],[0,0,1j,0], [0, 1j,0,0],[-1j,0,0,0]])
        elif n == 3:
            return np.matrix([[0,0,1,0],[0,0,0,-1],[-1,0,0,0],[0,1,0,0]])
        elif n == 5:
            return np.matrix([[1j,0,0,0], [0,1j,0,0], [0,0,-1j,0], [0,0,0,-1j]])
        else:
            raise ValueError('n must be 0, 1, 2, 3, or 5')
    elif repr == 'dirac':
        if n == 0:
            return np.matrix([[1,0,0,0], [0,1,0,0], [0,0,-1,0], [0,0,0,-1]])
        elif n in range(4):
            return gamma(n, repr = 'chiral')
        elif n == 5:
            return np.matrix([[0,0,1,0], [0,0,0,1], [1,0,0,0], [0,1,0,0]])
        else:
            raise ValueError('n must be 0, 1, 2, 3, or 5')
    elif repr == 'majorana':
        if n == 0:
            return np.kron(sigma(1),sigma(2))
        elif n == 1:
            return 1j*np.kron(sigma(0),sigma(3))
        elif n == 2:
            return -1j*np.kron(sigma(2),sigma(2))
        elif n == 3:
            return -1j*np.kron(sigma(0),sigma(1))
        elif n == 5:
            return np.kron(sigma(3),sigma(2))
        else:
            raise ValueError('n must be 0, 1, 2, 3, or 5')
    else:
        raise ValueError('repr must be \'chiral\', \'dirac\', or \'majorana\'')
    
#  define feynman slash
def slash(x : np.matrix, repr : str = 'chiral') -> np.matrix:
    '''
    ----------
    Parameters
    ----------
    x : 4D array-like
        The 4D vector to be slashed
    repr : str
        The representation of the gamma matrices. 'chiral', 'dirac', or 'majorana'. 
        Default is 'chiral'.
    ----------
    Output
    ----------
    The slashed vector in the specified representation.
    '''
    X = np.array(x)
    if X.shape != (1,4) and X.shape != (4,1):
        raise ValueError('x must be 4D array-like')
    if X.shape == (1,4):
        return X[0][0]*gamma(0, repr) + X[0][1]*gamma(1, repr) + X[0][2]*gamma(2, repr) + X[0][3]*gamma(3, repr)
    if X.shape == (4,1):
        return slash(x.T, repr)
    
# define the covering map of the (proper orthochronus) lorentz group
def cover(A : np.matrix) -> np.matrix:
    '''
    ----------
    Parameters
    ----------

    A : 2x2 matrix of determinant 1
        
    ----------
    Output
    ----------

    The proper orthochronous Lorentz group element corresponding to the 2x2 matrix A
    The correspondeng 4x4 matrix L = cover(A) is determined by the identity
    sigma(Lx) = A sigma(x) A^*
    '''
    if A.shape != (2,2):
        raise ValueError('A must be a 2x2 matrix')
    if np.linalg.det(A) != 1:
        warnings.warn(f'The determinant of {A} is not 1')
    L = np.array(np.zeros((4,4), dtype = complex))
    for i, j in itertools.product(range(4), range(4)):
            L[i,j] = sigma_inv( A * sigma(j) * A.H)[i,0]
    return np.matrix(L)

# define the covering of the compexified (proper) lorentz group
def cover_c(A : np.matrix, B: np.matrix) -> np.matrix:
    '''
    ----------
    Parameters
    ----------

    A : 2x2 matrix of determinant 1
    B : 2x2 matrix of determinant 1
        
    ----------
    Output
    ----------

    4x4 matrix:
        The proper complex Lorentz transformations corresponding to (A,B)
        L = cover_c(A,B) is determined by the identity
        sigma(Lx) = A sigma(x) B^T

    '''
    if A.shape != (2,2) or B.shape != (2,2):
        raise ValueError('A and B must be 2x2 matrices')
    if np.linalg.det(A) != 1 or np.linalg.det(B) != 1:
        warnings.warn(f'The determinants of {A} and {B} should be 1')
    L = np.array(np.zeros((4,4), dtype = complex))
    for i, j in itertools.product(range(4), range(4)):
            L[i,j] = sigma_inv( A * sigma(j) * B.T)[i,0]
    return np.matrix(L)



In [175]:
# define correspondence between R^3 and mass hyperboloid
def hyp_to_r3(f : FunctionType, m : float) -> FunctionType:
    '''
    ----------
    Parameters
    ----------
    f : function
        A function on upper hyperboloid sheet of mass m
    m : float
        The mass of the particle

    ----------
    Output
    ----------
    The function on R^3 corresponding to f

    '''
    return lambda x1,x2,x3: f(np.sqrt(m**2 + x1**2+ x2**2+ x3**2),x1,x2,x3) 


# define the Hilbert space for the free scalar field
def inner_free(f : FunctionType, g : FunctionType, m : float) -> float:
    '''
    ----------
    Parameters
    ----------
    f, g : functions
        square-summable functions on R^4

    ----------
    Output
    ----------
    The inner product of f and g with respect to the
    Hilbert space of the free scalar field

    '''
    F = hyp_to_r3(f,m); G = hyp_to_r3(g,m)
    h = lambda x1,x2,x3: 1/(2*np.sqrt(m**2 + x1**2 + x2**2 + x3**2))* np.conjugate(F(x1,x2,x3)) * G(x1,x2,x3)
    return spi.tplquad(h, -np.inf, np.inf, -np.inf, np.inf, -np.inf, np.inf)[0]
    

In [176]:
f = lambda t,x,y,z: np.exp(-t**2 - x**2 - y**2 - z**2)
inner_free(f,f,1)

0.04079673644123803